In [2]:
import rdkit
from pathlib import Path

path_to_refined = "pdbbind/refined-set/"
path_to_general = "pdbbind/v2020-other-PL"

In [13]:
def extract_mol2_pdb_id(mol2_file):
    pdb_id = None
    ligand_name = None
    with open(mol2_file, 'r') as f:
        for line in f:
            if line.startswith("@<TRIPOS>MOLECULE"):
                ligand_name = next(f).strip()  # The next line is usually the ligand name
                pdb_id = ligand_name.split("_")[0]  # If named like "4HWP_ligand", extract "4HWP"
                break
    return pdb_id

In [1]:
from rdkit import Chem
from Bio import PDB

all_smiles = []
all_sequences = []
parser = PDB.PDBParser(QUIET=True)
aa_map = { 
        "ALA": "A", "ARG": "R", "ASN": "N", "ASP": "D", "CYS": "C",
        "GLN": "Q", "GLU": "E", "GLY": "G", "HIS": "H", "ILE": "I",
        "LEU": "L", "LYS": "K", "MET": "M", "PHE": "F", "PRO": "P",
        "SER": "S", "THR": "T", "TRP": "W", "TYR": "Y", "VAL": "V"
    }

for subdir in Path(path_to_refined).iterdir():
    if subdir.is_dir():
        mol2_file = next(subdir.glob("*.mol2"), None)
        protein_file = next(subdir.glob("*protein.pdb"), None)
        if mol2_file:
            pdb_id = extract_mol2_pdb_id(mol2_file)
            mol = Chem.MolFromMol2File(str(mol2_file), removeHs=False)
            if mol:
                smiles = Chem.MolToSmiles(mol)
                all_smiles.append(smiles)
        if protein_file:
            structure = parser.get_structure("protein", protein_file)
            sequence = ""
            m = 0
            c = 0
            for model in structure:
                m += 1
                for chain in model:
                    c += 1
                    for residue in chain:
                        res_name = residue.get_resname()
                        if res_name in aa_map:
                            sequence += aa_map[res_name]
                        else:
                            print(f"Missing residue {res_name}")
            if m > 1:
                print("Protein with 2+ models")
            if c > 1:
                print("Protein with 2+ chains")
            all_sequences.append(sequence)


In [4]:
structure = parser.get_structure("protein", "pdbbind/v2020-other-PL/11gs/11gs_protein.pdb")

In [5]:
from Bio.SeqUtils import seq1
chains = {chain.id:seq1(''.join(residue.resname for residue in chain)) for chain in structure.get_chains()}

In [9]:
chains

{'A': 'PYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKASCLYGQLPKFQDGDLTLYQSNTILRHLGRTLGLYGKDQQEAALVDMVNDGVEDLRCKYISLIYTNYEAGKDDYVKALPGQLKPFETLLSQNQGGKTFIVGDQISFADYNLLDLLLIHEVLAPGCLDAFPLLSAYVGRLSARPKLKAFLASPEYVNLPINGNGKQ',
 'B': 'PYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKASCLYGQLPKFQDGDLTLYQSNTILRHLGRTLGLYGKDQQEAALVDMVNDGVEDLRCKYISLIYTNYEAGKDDYVKALPGQLKPFETLLSQNQGGKTFIVGDQISFADYNLLDLLLIHEVLAPGCLDAFPLLSAYVGRLSARPKLKAFLASPEYVNLPINGNGKQ',
 ' ': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'}